In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from pipeline_utils import (
    COMPUTE, CONCURRENT_TRIALS, DELAY_EVALUATION, EVALUATION_INTERVAL, GOAL, METRIC, SAMPLING_ALGORITHM, TIMEOUT, TIMEOUT_PLUS, TOTAL_TRIALS, #SWEEP
    SUBSCRIPTION, RESOURCE_GROUP, WS_NAME,  #AUTHENTICATE
    GBC_PATH, NBC_PATH, RFC_PATH, SVC_PATH, XGB_PATH,    #COMPONENTS PATHS
    GINI_PATH,    #FEAT SEL PATHS
    PREP_DATA_PATH,  #PREP DATA PATH
    GBC_BY_GINI, NBC_BY_GINI, RFC_BY_GINI, SVC_BY_GINI, XGB_BY_GINI,    #PIPELINES
    GINI,   #FEAT SEL
    get_experiment_names,   #FUNCTIONS
)

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [2]:
TRAIN_DATAS = [
    "vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017.csv"
]

TEST_DATAS = [
    "vrex_encoded_tf_idf_2018_2019_2020_2021.csv"
]

version = "v1"

arr_data_to_train = []
arr_data_to_test = []

for to_train, to_test in zip(TRAIN_DATAS, TEST_DATAS):
    data_to_train = ml_client.data.get(name=to_train.split(".")[0], version=version)
    arr_data_to_train.append(data_to_train)
    print(f"Data to train asset URI: {data_to_train.path} - name: {to_train.split('.')[0]}")

    data_to_test = ml_client.data.get(name=to_test.split('.')[0], version=version)
    arr_data_to_test.append(data_to_test)
    print(f"Data to test asset URI: {data_to_test.path} - name: {to_test.split('.')[0]}")

Data to train asset URI: azureml://subscriptions/a3f56f48-3efb-4970-81a3-e4eda598333c/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/3cfdebd258ea2db7363c55bb841ca887/vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017.csv - name: vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017
Data to test asset URI: azureml://subscriptions/a3f56f48-3efb-4970-81a3-e4eda598333c/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/3992df39ce41e34367e629d6656f4e9f/vrex_encoded_tf_idf_2018_2019_2020_2021.csv - name: vrex_encoded_tf_idf_2018_2019_2020_2021


In [3]:
# importing the Component Package
from azure.ai.ml import load_component

data_prep_component = load_component(source=PREP_DATA_PATH)
data_prep_component = ml_client.create_or_update(data_prep_component)

feat_sel_component = load_component(source=GINI_PATH)
feat_sel_component = ml_client.create_or_update(feat_sel_component)

train_gbc = load_component(source=GBC_PATH)
train_gbc = ml_client.create_or_update(train_gbc)

train_nbc = load_component(source=NBC_PATH)
train_nbc = ml_client.create_or_update(train_nbc)

train_rfc = load_component(source=RFC_PATH)
train_rfc = ml_client.create_or_update(train_rfc)

train_svc = load_component(source=SVC_PATH)
train_svc = ml_client.create_or_update(train_svc)

train_xgb = load_component(source=XGB_PATH)
train_xgb = ml_client.create_or_update(train_xgb)

In [4]:

# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy

In [5]:
@dsl.pipeline(
    name=GBC_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_gbc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_percentage=0.5,
    )

    train_gbc_job = train_gbc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,  
        n_estimators_to_gbc=Choice(values=[50, 100, 200]),
        learning_rate_to_gbc=Uniform(min_value=0.01, max_value=0.3),
    )

    sweep_step_to_gbc = train_gbc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_gbc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)

    sweep_step_to_gbc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)


In [6]:
@dsl.pipeline(
    name=NBC_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_nbc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_percentage=0.5,
    )

    train_nbc_job = train_nbc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,  
    )

In [7]:
@dsl.pipeline(
    name=RFC_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_rfc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_percentage=0.5,
    )

    train_rfc_job = train_rfc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel, 
        n_estimators_to_rfc=Choice(values=[50, 100, 200]),
    )

    sweep_step_to_rfc = train_rfc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_rfc.set_limits(max_total_trials=TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)
    sweep_step_to_rfc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [8]:
@dsl.pipeline(
    name=SVC_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_svc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_percentage=0.5,
    )

    train_svc_job = train_svc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,   
        kernel_to_svc=Choice(values=["linear", "rbf", "poly", "sigmoid","precomputed"]),
        gamma_to_svc=Choice(values=["scale", "auto"]),
    )

    sweep_step_to_svc = train_svc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_svc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT_PLUS)
    sweep_step_to_svc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [9]:
@dsl.pipeline(
    name=XGB_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_xgb_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_percentage=0.5,
    )

    train_xgb_job = train_xgb(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,   
        n_estimators_to_xgb=Choice(values=[100, 500, 1000]),
        learning_rate_to_xgb=Uniform(min_value=0.01, max_value=0.3),
    )

    sweep_step_to_xgb = train_xgb_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_xgb.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT_PLUS)
    sweep_step_to_xgb.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [10]:
pipelines = []

for data_to_train, data_to_test in zip(arr_data_to_train, arr_data_to_test):

    pipelines.append(train_nbc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_gbc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_rfc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_svc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_xgb_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))


In [12]:
experiment_names = get_experiment_names(TRAIN_DATAS, TEST_DATAS, GINI)
for pipeline, experiment_name in zip(pipelines, experiment_names):
    pipeline_job = ml_client.jobs.create_or_update(
        pipeline,
        experiment_name=experiment_name,
    )
    ml_client.jobs.stream(pipeline_job.name)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_GaussianNB_2024_06_05_04_03_50
vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_GradientBoostingClassifier_2024_06_05_04_03_50
vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_RandomForestClassifier_2024_06_05_04_03_50
vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_SVC_2024_06_05_04_03_50
vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_XGBoost_2024_06_05_04_03_50
RunId: loving_caravan_x7v1h0kjw3
Web View: https://ml.azure.com/runs/loving_caravan_x7v1h0kjw3?wsid=/subscriptions/a3f56f48-3efb-4970-81a3-e4eda598333c/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-experiments

Str